In [ ]:
# 개인정보에 따른 구매여부
# 나이, 경력, 연봉에 따라서 구매를 했는지 안 했는지 확인
# scale을 활용하여 train / test 데이터 분리
# scv

In [ ]:
# train data open
import pandas as pd

# 훈련용 데이터만으로 전처리 기준을 만든다
df = pd.read_csv('./data/train_dataset.csv')
df_raw = df.copy()  # 원본 데이터 복사본 생성
df.head()

In [ ]:
# data scaling
from sklearn.preprocessing import StandardScaler

# 사용할 컬럼 이름을 미리 정리
feature_cols = ['Age', 'Experience(Years)', 'Salary(만원)']
target_col = 'Purchased'

# 스케일러는 train 데이터로만 fit
scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])

df.head()

In [ ]:
# target encoding (문자형일 때만 인코딩)
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
if df[target_col].dtype == 'object':
    df[target_col] = le.fit_transform(df[target_col])
else:
    # 숫자형이면 그대로 사용하되, 나중에 test 변환을 위해 classes_ 등록
    le.fit(df[target_col])

df.head()

In [ ]:
# 최종 데이터셋
df_final = df
df_final

In [ ]:
# 데이터 학습용 입력/정답 분리
X_train = df[feature_cols]
y_train = df[target_col]
X_train.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# train 데이터 전체로 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# test 데이터를 열어 예측 및 평가하기
df_test = pd.read_csv('./data/test_dataset.csv')
df_test.head()


In [ ]:
# test 데이터도 동일하게 전처리 (train에서 fit한 scaler 사용)
feature_test_scaled = scaler.transform(df_test[feature_cols])
feature_test_scaled

In [ ]:
# df_test 에 적용
df_test[feature_cols] = feature_test_scaled
df_test.head()

In [ ]:
# 예측
X_new = df_test[feature_cols]
X_new

In [ ]:
y_new_pred = model.predict(X_new)
y_new_pred

In [ ]:
# 예측값 컬럼 추가
df_test['Purchased_Predicted'] = y_new_pred
df_test

In [ ]:
df_test[['Age', 'Experience(Years)', 'Salary(만원)', 'Purchased_Predicted']]

In [ ]:
# accuracy 평가 (test 데이터의 실제 값과 비교)
from sklearn.metrics import accuracy_score, classification_report

if df_test[target_col].dtype == 'object':
    y_test = le.transform(df_test[target_col])
else:
    y_test = df_test[target_col].to_numpy()

accuracy = accuracy_score(y_test, y_new_pred)
report = classification_report(y_test, y_new_pred)
accuracy, report